
# 1. 1 - Importando bibliotecas

In [ ]:
from zipfile import ZipFile

import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import folium
from geopy.geocoders import Nominatim
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from folium.plugins import MarkerCluster
from folium import plugins

from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from math import sqrt
from scipy.stats import randint

# machine learning
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedKFold


import warnings
warnings.filterwarnings("ignore")

# 1.2 Funções aplicadas neste notebook:

In [ ]:
def descompacta_zip(arq_descompactar):
    with ZipFile (arq_descompactar, 'r') as zip:
        zip.extractall()
        zip.printdir()
    return arq_descompactar+' Descompactado'
	

def extract_data_bairros(data):
    bairros = []
    for i in data:
        bairros.append(i.split('\n'))
    for i in bairros:
        if len(i)<4:
            bairros.remove(i)
    
    Distritos=[]
    Area=[]
    População=[]
    Densidade_Demográfica=[]
    for i in bairros:
        if i[-4].find('TOTAL')==-1:
            Distritos.append(i[-4])
            Area.append(i[-3])
            População.append(i[-2])
            Densidade_Demográfica.append(i[-1])
            
    len(Distritos),len(Area),len(População),len(Densidade_Demográfica)
    if len(Distritos)==len(Area)==len(População)==len(Densidade_Demográfica):
        n = len(Distritos)
        print('Extração de dados está CORRETA\nCada lista possui',n, 'elementos')
        
    else:
        print('Extração de dados NÃO está correta!')
        
    # Criação do Dataframe
    
    colunas = ['Distrito', Area[0],População[0],Densidade_Demográfica[0],]
    
    df = pd.DataFrame(list(zip(Distritos, Area,População,Densidade_Demográfica)), columns = colunas)
    df.drop(index=0, inplace=True)
    return df


#Função converte colunas numéricas de um Dataframe para logaritno natural: Precisa informar colunas
def simetric(df, cols):
    cols_ln=[]
    for col in cols:
        sk = df[col].skew()
        if (sk>1 or sk<-1):
            print('Assimetria (Skewness) em %s é %.2f' %(col,sk))
            nova_coluna = col+'_ln'
            df[nova_coluna]=(np.log(df[col])).replace(-np.inf, 0)
            sk_ln = df[nova_coluna].skew()
            print('Assimetria (Skewness) em %s é %.2f' %(nova_coluna,sk_ln))
            if sk_ln > sk:
                df.drop(columns = nova_coluna, inplace = True)
                print('Coluna %s Não alterada\n' %(col))
            else:
                df[col]=df[nova_coluna]
                df.drop(columns = nova_coluna, inplace = True)
#                cols_ln.append(nova_coluna)
                
                print('Coluna %s ajustada \n' %(col))

                
    return cols_ln


# Função para calcular limites máximos e mínimos para outliers
def outlier_iqr(df, cols):
    for i in cols:
        df.sort_values(by=i, ascending=True, na_position='last')
        q1, q3 = np.nanpercentile(df[i], [25,75])
        iqr = q3-q1
        lower_bound = q1-(1.5*iqr)
        upper_bound = q3+(1.5*iqr)
        outlier_data = df[i][(df[i] < lower_bound) | (df[i] > upper_bound)] #creating a series of outlier data
        perc = (outlier_data.count()/df[i].count())*100
        print('Outliers em %s %.2f%% Total de registros: %.f' %(i, perc, outlier_data.count()))
    return lower_bound, upper_bound



#Cria um scatter plot com o resultado do modelo de machine learning.
def plotar_grafico(y_t,y_pred):
    # Visualizando as diferenças entre preços atuais e preços preditos
    sns.set(font_scale = 1.2)
    plt.figure(figsize = (8,5))   
    plt.scatter(y_t, y_pred)
    range = [y_t.min(), y_pred.max()]
    plt.plot(range, range, 'blue')
    plt.xlabel("Preços")
    plt.ylabel("Preços preditos")
    plt.title("Preços vs Preços preditos")
    plt.show()
    
    # Checando resíduos
    sns.set(font_scale = 1.2)
    plt.figure(figsize = (8,5))
    plt.scatter(y_pred,y_t-y_pred)
    plt.plot([y_pred.min(),y_pred.max()],[0,0], 'blue')
    plt.title("Predicões vs resíduos")
    plt.xlabel("Predicões")
    plt.ylabel("Resíduos")
    plt.show()
    
    # Checando normalidade dos erros
    sns.set(font_scale = 1.2)
    plt.figure(figsize = (8,5))
    sns.distplot(y_t-y_pred)
    plt.title("Histograma dos Resíduos")
    plt.xlabel("Resíduos")
    plt.ylabel("Frequência")
    plt.show()
    return

# Coleta dos dados
## Primeiro Dataframe - df_imoveis

In [ ]:
# Chamando função para descompactar .zip
descompacta_zip('sao-paulo-properties-april-2019.csv.zip')

In [ ]:
# Abertura de arquivo e visualização dos primeiros registros:

csv_file = 'sao-paulo-properties-april-2019.csv'

df = pd.read_csv(csv_file, sep=',')

del csv_file
display(df.head())
df.info()

In [ ]:
# Checando valores únicos para tipo de negociação (será extraído apenas os registros de venda)

print('Tipo de negociação:', list(df['Negotiation Type'].unique()))

In [ ]:
# Seleção dos imóveis para venda:
df_imoveis = df.loc[df['Negotiation Type']== 'sale']
df_imoveis.drop(columns=['Negotiation Type'], inplace=True)

# Exclusão de dados duplicados, renomeação das colunas e atualização do index
df_imoveis = df_imoveis.drop_duplicates()
df_imoveis.rename(columns={'Price': 'Preco', 'Size':'Area', 'Rooms': 'Quartos', 'Toilets':'Banheiros',
                           'Parking': 'Garagem', 'Elevator': 'Elevador', 'Furnished': 'Mobiliado', 'Swimming Pool': 'Piscina',
                           'New': 'Novo', 'District': 'Distrito', 'Property Type':'Tipo_imovel'}, inplace=True)

# Como verificado, a coluna "Distrito" contém nome do distrito e cidade.
# Fatiamento dessa coluna em 'Distrito' e 'Cidade': 
df_imoveis[['Distrito','Cidade']] = df_imoveis['Distrito'].str.split('/', expand = True)
df_imoveis.reset_index(drop=True, inplace=True)

# Verificação se todos os registros são da cidade de São Paulo e quais os tipos de imóveis que compões o DataFrame:
print(' Cidades no dataframe:', list(df_imoveis['Cidade'].unique()), '\n',
      'Tipos de imóveis: ', list(df_imoveis['Tipo_imovel'].unique()), '\n')

del df
df_imoveis.info()

In [ ]:
# Como os registros são únicos, essas colunas serão excluídas

df_imoveis.drop(columns=['Cidade','Tipo_imovel'], inplace=True)
df_imoveis.head()

In [ ]:
pd.options.display.float_format = '{:.4f}'.format

In [ ]:
# Descrição sumária dos registros do Dataframe:
display(df_imoveis.round().describe())
# Checagem de valores Nan:
print('\n',"Total de valores NaN: ", df_imoveis.isna().sum().sum())


In [ ]:
# Visualização da distribuição dos dados do Dataframe

df_hist = df_imoveis[['Preco', 'Condo', 'Area', 'Quartos', 'Banheiros', 'Suites', 'Garagem']]

fig, axs = plt.subplots(ncols=2, nrows=4, figsize=(12, 8))
index = 0
axs = axs.flatten()
for k,v in df_hist.items():
    sns.histplot(v, ax=axs[index])
    index += 1
plt.tight_layout(pad=0., w_pad=0.5, h_pad=5.0)

del df_hist

## Segundo Dataframe - df_bairros

In [ ]:
# Função para buscar dados dos bairros e subprefeituras da cidade de São Paulo:
# <'https://www.prefeitura.sp.gov.br/cidade/secretarias/subprefeituras/subprefeituras/dados_demograficos/index.php?p=12758'>

# Uso de Beautiful Soup

url = 'https://www.prefeitura.sp.gov.br/cidade/secretarias/subprefeituras/subprefeituras/dados_demograficos/index.php?p=12758'
html_text = requests.get(url).text
soup = BeautifulSoup(html_text, 'html.parser')
title = soup.find('tbody').get_text()
data=title.strip().split('\n\n')
data

In [ ]:
# Criação do Dataframe df_bairros

df_bairros = extract_data_bairros(data)

df_bairros.to_csv("df_bairros.csv")

type(df_bairros)

In [ ]:
#df_bairros = pd.read_csv('bairros_SP.csv', index_col =0)
df_bairros.info()

In [ ]:
# Criação do Dataframe df_bairros      
display(df_bairros.head())
display(df_bairros.tail())

In [ ]:
# Uso de função lambda e .replace para mudar vírgulas por pontos e pontos e vírgulas por espaço vazio

df_bairros['Área (km²)'] = df_bairros['Área (km²)'].apply(lambda x: float(x.replace(",",".")))
df_bairros['População (2010)'] = df_bairros['População (2010)'].apply(lambda x: float(x.replace(".","")))
df_bairros['Densidade Demográfica (Hab/km²)'] = df_bairros['Densidade Demográfica (Hab/km²)'].apply(lambda x: float(x.replace(",","").replace(".","")))

# Visualizando as 'pontas' do Dataframe
display(df_bairros.head())
display(df_bairros.tail())

In [ ]:
##df_bairros = pd.read_csv('bairros_SP.csv', index_col=0)

In [ ]:
# Descrição sumária do Dataframe:
display(df_bairros.info())
display(df_bairros.describe())
print('\n',"Valores NaN:",'\n\n', df_bairros.isna().sum())

In [ ]:
# Visualização da distribuição dos dados do Dataframe
df= df_bairros[['Área (km²)','População (2010)', 'Densidade Demográfica (Hab/km²)']]

fig, axs = plt.subplots(ncols=3, nrows=1, figsize=(20, 5))
index = 0
axs = axs.flatten()
for k,v in df.items():
    sns.histplot(v, ax=axs[index])
    index += 1
plt.tight_layout(pad=0.2, w_pad=0.3, h_pad=3.0)

## Terceiro Dataframe - df_rendas

In [ ]:
# Abrindo arquivo .xlsx e renomeando colunas

df_rendas = pd.read_excel('Domicilios_faixa_rendimento_sal_minimos_2010.xls', skiprows=6, skipfooter=5)

df_rendas.rename(columns = {'Unnamed: 0' : 'Distrito', 'Unnamed: 1': 'Total_domicilios',
                            "Sem rendimento (3)": "Sem_rendimento"}, inplace=True)

# Renomeando a última coluna:
cols = df_rendas.columns
for col in cols:
    if col.startswith('Sem'):
        df_rendas.rename(columns = {col:"Sem_rendimento"}, inplace=True)
df_rendas.columns

In [ ]:
# Visualizando as 'pontas' do Dataframe
display(df_rendas.head())
display(df_rendas.tail())

In [ ]:
# Descrição sumária do Dataframe:
display(df_rendas.info())
display(df_rendas.describe())
df_rendas.isna().sum()

In [ ]:
#Correção Registro Distrito 'São Miguel' e 'Moóca'

df_rendas.iloc[[105],[0]]='São Miguel'

df_rendas.loc[df_rendas['Distrito'] == 'Moóca', 'Distrito']='Mooca'

In [ ]:
# Exclusão de registros manualmente.
#Ex.: 2 Registros Butantã: um é o distrito e o outro se refere a Subprefeitura
index_drop= [0,1,5,11,15,19,23,28,31,34,37,41,44,49,51,54,61,64,71,74,79,82,87,91,95,99,103,107,116,120,124]

df_rendas.drop(index=index_drop, inplace=True)
df_rendas.drop_duplicates(inplace=True)
# Correção de pontos e vírgulas para números e conversão para percentagem
cols = df_rendas.columns

for col in cols:
    if (col != 'Distrito'):
        df_rendas[col] = df_rendas[col].apply(lambda x: float(x.replace(".","")))

print("Total de registros df_rendas: ",len(list(df_rendas['Distrito'])))

In [ ]:
# Conversão para percentagem
        
for col in cols:
    if (col != 'Distrito') and (col != 'Total_domicilios'):
        df_rendas[col+'_SM_%'] = round(df_rendas[col]/df_rendas['Total_domicilios']*100,2)
        df_rendas.drop(columns=col, inplace=True)
                

In [ ]:
# Descrição sumária do Dataframe e checagem de valores Nan:
display(df_rendas.info())
display(df_rendas.describe())
print("Total de valores NaN: ", df_rendas.isna().sum().sum())

# Primeira junção de Dataframes:

### df_bairros e df_rendas

In [ ]:
# Merge dos Dataframes df_bairros e df_rendas, inner join e contagem do número de registros
df_distritos = pd.merge(df_bairros, df_rendas, on='Distrito', how='inner')
print(' Total de registros em df_distritos: ', len(df_distritos), '\n',
      'Total de registros em df_bairros:   ',len(df_bairros),'\n',
      'Total de registros em df_rendas:   ',len(df_rendas))

# Checar Nan's nas colunas e percentual de Nan's em cada coluna
print(" Total de Valores NaNs:               ", df_distritos.isna().sum().sum())

In [ ]:
display(df_distritos.head())
df_distritos.tail()

# Segunda junção de Dataframes:

### df_imoveis e df_distritos

In [ ]:
# Merge Dataframes df_imoveis e df_distritos e posterior verificação pois a chave "Distrito" em df_imoveis pode conter erros

df = pd.merge(df_imoveis, df_distritos, on = 'Distrito', how = 'left')
l_distr = df['Distrito'].loc[df['População (2010)'].isna()].unique()
print(' Total de Distritos não identificados:', len(l_distr),'\n','Distritos não identificados: ', l_distr)

In [ ]:
# Correção gramatical dos nomes dos distritos

df_distritos.loc[df_distritos['Distrito'] == 'Guaianases', 'Distrito']='Guaianazes'

df_imoveis.loc[df_imoveis['Distrito'] == 'Jardim São Luis', 'Distrito']='Jardim São Luís'
df_imoveis.loc[df_imoveis['Distrito'] == 'Medeiros', 'Distrito']='Vila Medeiros'

# correção de lugares:df_imoveis com local identificado pelo nome da vila ou bairro e df_distritos pelo nome oficial
df_imoveis.loc[df_imoveis['Distrito'] == 'Vila Madalena', 'Distrito']='Pinheiros'
df_imoveis.loc[df_imoveis['Distrito'] == 'Brooklin', 'Distrito']='Itaim Bibi'
df_imoveis.loc[df_imoveis['Distrito'] == 'Vila Olimpia', 'Distrito']='Itaim Bibi'


In [ ]:
# Merge com adequação dos nomes dos Distritos

df = pd.merge(df_imoveis, df_distritos, on = 'Distrito', how = 'left')

l_distr = df['Distrito'].loc[df['População (2010)'].isna()].unique()
print(' Total de Distritos não identificados:', len(l_distr),'\n','Distritos não identificados: ', l_distr)

df.dropna(inplace=True)
df.reset_index(drop = True,inplace=True)

print( " Total de valores nulos:", df.isna().sum().sum(),'\n Dimensão df:', df.shape)

In [ ]:
# Deletando demais dataframes:

del df_bairros
del df_rendas
del df_imoveis
#del df_distritos
del url
del html_text
del soup 
del title
del data

In [ ]:
df.to_csv('df_SP_28-03-22.csv')

# Análise Exploratória de Dados

In [ ]:
df.info()

In [ ]:
# Visualização da distribuição dos dados do Dataframe

l_columns = list(df.columns[0:7])+ list(df.columns[14:])
sns.set(font_scale = 1.5)
hist = df[l_columns].hist(bins=15, figsize=(15,20))

In [ ]:
# Continuando a exploração dos dados: Relação do Preço com as variáveis: elevador, mobiliado, piscina e novo
l_columns = df.columns[7:11]
pos = 0
plt.figure(figsize = (10,10))
for i in l_columns:
    pos +=1
    plt.subplot(2,2,pos) 
    ax = sns.boxenplot(x = i , y = 'Preco',data = df )

In [ ]:
# Visualização Relação de Preço do imóvel em relação à: Condomínio, Área, Quartos, Banheiros, Suítes e Garagem
pd.options.display.float_format = '{:.2f}'.format
l_columns = df.columns[1:7]
sns.set(font_scale = 1.2)
pos = 0
plt.figure(figsize = (15,8))
for i in l_columns:
    pos +=1
    plt.subplot(2,3,pos)    
    ax = sns.regplot(x = i, y = 'Preco', data = df)

In [ ]:
# Continuando a exploração dos dados: Relação do Preço com as demais variáveis advindas dos dataset df_distritos:

l_columns = df.columns[14:]
pos = 0
plt.figure(figsize = (14,16))
for i in l_columns:
    pos +=1
    plt.subplot(4,3,pos)    
    ax = sns.regplot(x = i, y = 'Preco', data = df, x_bins=50)
    

In [ ]:
# Variáveis de localização: Latitude e Longitude
# É esperado que formem um desenho
ax = sns.pairplot(df, y_vars = 'Latitude', x_vars = ['Longitude'],
                  height = 4, kind = 'reg')

ax.fig.suptitle('Dispersão entre as Variáveis', fontsize=10, y=0.5)

In [ ]:
# Distribuição dos imóveis na cidade de São Paulo

grouped = df.groupby('Distrito')['Preco'].mean().reset_index()

grouped.rename(columns={'Preco':'Preco_Medio'}, inplace=True)

grouped = grouped.sort_values(by = 'Preco_Medio', ascending = False).head(5)
plt.figure(figsize = (5,4))
sns.set(font_scale = 1.40)
ax = sns.barplot(x='Preco_Medio', y='Distrito', data = grouped,palette = 'plasma')
print(" Distritos com os maiores preço médio de apartamentos: ")

In [ ]:
# Distribuição dos imóveis na cidade de São Paulo

grouped = df.groupby('Distrito')['Preco'].mean().reset_index()


grouped.rename(columns={'Preco':'Preco_Medio'}, inplace=True)

grouped = grouped.sort_values(by = 'Preco_Medio', ascending = True).head(5)
plt.figure(figsize = (5,4))
sns.set(font_scale = 1.40)
ax = sns.barplot(x='Preco_Medio', y='Distrito', data = grouped,palette = 'plasma')
print(" Distritos com os menores preço médio de apartamentos: ")

In [ ]:
# Distribuição dos imóveis na cidade de São Paulo

grouped = df.groupby('Distrito')['Preco'].mean().reset_index()

grouped = grouped.loc[grouped['Preco']>1000000].sort_values(by = 'Preco',ascending = False)
plt.figure(figsize = (10,10))
sns.set(font_scale = 1.4)
ax = sns.barplot(x='Preco', y='Distrito', data = grouped,palette = 'plasma')
print("Total de distritos com preço médio acima de 1 milhão de reais: ", len(grouped))

In [ ]:
# Distribuição dos imóveis na cidade de São Paulo

grouped = df.groupby('Distrito')['Preco'].mean().reset_index()

grouped = grouped.loc[grouped['Preco']<250000].sort_values(by = 'Preco',ascending = False)
plt.figure(figsize = (10,10))
sns.set(font_scale = 1.4)
ax = sns.barplot(x='Preco', y='Distrito', data = grouped,palette = 'plasma')
print("Total de distritos com preço médio abaixo de 250 mil de reais: ", len(grouped))

In [ ]:
# Dicionário para gerar Nuvem de Palavras (worldcloud)

#from PIL import Image
# Import image to np.array
#mask = np.array(Image.open('SP_mapas.png'))

data_cloud = dict(df['Distrito'].value_counts())

# gerar uma wordcloud
wordcloud = WordCloud(background_color="white", colormap = "Greens_r",
                      width=1600, height=1000, max_words=93,
                      max_font_size=250,
                      min_font_size=3).generate_from_frequencies(data_cloud)
 
# mostrar a imagem final
fig, ax = plt.subplots(figsize=(14,14))
ax.imshow(wordcloud, interpolation='bilinear')
ax.set_axis_off()
 
plt.imshow(wordcloud);
#wordcloud.to_file("imóveis_summary.png")

In [ ]:
# Distribuição dos imóveis na cidade de São Paulo

grouped = df.groupby('Distrito')['População (2010)'].mean().reset_index()

grouped = grouped.loc[grouped['População (2010)']>200000].sort_values(by = 'População (2010)',
                                                                                      ascending = False)
plt.figure(figsize = (8,12))
sns.set(font_scale = 3.39)
ax = sns.barplot(x='População (2010)', y='Distrito', data = grouped,palette = 'plasma')
print("Total de distritos com Densidade Demográfica (Hab/km²) abaixo de 5000 hab/km²: ", len(grouped))

In [ ]:
# Distribuição dos imóveis na cidade de São Paulo

grouped = df.groupby('Distrito')['Mais de 20_SM_%'].mean().reset_index()

grouped = grouped.loc[grouped['Mais de 20_SM_%']>25].sort_values(by = 'Mais de 20_SM_%',
                                                                                      ascending = False)
plt.figure(figsize = (8,12))
sns.set(font_scale = 3.39)
ax = sns.barplot(x='Mais de 20_SM_%', y='Distrito', data = grouped,palette = 'plasma')
print("Total de distritos com Densidade Demográfica (Hab/km²) abaixo de 5000 hab/km²: ", len(grouped))

In [ ]:
# Uso biblioteca folium usando colunas Latitudes e Longitudes informadas:

#from folium.plugins import MarkerCluster

m = folium.Map(
    location=[-23.3, -46.6],
    tiles='Stamen Toner',
    zoom_start=8
)
mc = MarkerCluster()
    
for index, value in df.iterrows():
  mc.add_child(folium.Marker([value['Latitude'], value['Longitude']], 
              popup=str(value['Distrito']),
              tooltip=value['Distrito'],
              icon=folium.Icon(icon='book'))).add_to(m) 
m

# Tratamento dados NaN
### Tratamento Valor de condomínio

In [ ]:
# A princípio não há dados nulos

print('Total de dados nulos: ', df.isna().sum().sum())

In [ ]:
# Visualizando valores muito baixos, há valores 0 para condomínio.
# E valores muito próximos de Zero

cond_0 = len(df[df['Condo']==0])

prox_0 = len(df[df['Condo']<=10].loc[df['Condo']>0])

print('Total de registros com Condomínio igual a zero: ', cond_0)

print('Total de registros com Condomínio próximo a zero: ', prox_0)

In [ ]:
# Substituindo valores iguais ou menores que 10 por None:

df['Condo'] = np.where(df['Condo']<= 10,np.nan, df['Condo'])

# substituir o missing pela mediana da coluna
df['Condo'].fillna(round(df['Condo'].median()), inplace=True)


df.Condo.describe()

In [ ]:
df['Condo'] = df['Condo'].astype('int64')

## Tratamento Valores de Latitude e Longitude

In [ ]:
# Verificando valores iguais a zero:

lat_zero = len(df[['Distrito','Latitude', 'Longitude']].loc[df['Latitude']==0])
long_zero = len(df[['Distrito','Latitude', 'Longitude']].loc[df['Longitude']==0])

print('Total de registros de Latitude iguais a zero: ',lat_zero)
print('Total de registros de Longitude iguais a zero: ',long_zero)

In [ ]:
# Uso Biblioteca Nominatim para identificar a Latitude e Longitude Máxima e Mínima da cidade de São Paulo 

place = 'São Paulo, Região Imediata de São Paulo, Região Metropolitana de São Paulo,'

geolocator = Nominatim(user_agent="geolocalização")
d_lat_sp={}
d_long_sp={}
location = geolocator.geocode( place)
d_lat_sp['São Paulo']=[float(location.raw['boundingbox'][0]),float(location.raw['boundingbox'][1])]
d_long_sp['São Paulo']= [float(location.raw['boundingbox'][2]), float(location.raw['boundingbox'][3])]

print('Latitudes Mínima e Máxima:', d_lat_sp)
print('Longitudes Mínima e Máxima:',d_long_sp )

In [ ]:
# Substituindo os registros de posição geográfica incorretos por NaN(fora dos limites máximos e mínimos)

df['Latitude'] = np.where(df['Latitude']>=d_lat_sp['São Paulo'][0],
                                          np.where(df['Latitude']<=d_lat_sp['São Paulo'][1],
                                         df['Latitude'],
                                          np.nan),np.nan)

df['Longitude'] = np.where(df['Longitude']>=d_long_sp['São Paulo'][0],
                                          np.where(df['Longitude']<=d_long_sp['São Paulo'][1],
                                         df['Longitude'],
                                          np.nan),np.nan)

In [ ]:
# A partir do df, cria outros dois dataframes com as médias das latitudes e longitudes para cada distrito.
# Com merge cria-se novo df (df_medias) com as medias e elimina-se os valor nulos 
# Assim obtêm-se a média de latitudes e longitudes em cada distrito para substituir os registros nulos:

df_media_lat = df[['Distrito', 'Latitude']].groupby(['Distrito']).mean().reset_index()
df_media_long = df[['Distrito', 'Longitude']].groupby(['Distrito']).mean().reset_index()

df_medias = pd.merge(df_media_lat, df_media_long, on = 'Distrito', how = 'inner')
df_medias.dropna(inplace=True)
print('Dimensão do DataFrame df_medias: ', df_medias.shape)
display(df_medias.head())

In [ ]:
# Verificando quais os distritos não contêm posição geográfica subtraindo-os do df_medias no df,
# resultando numa relação de distritos sem as médias das posições geográficas.

places = list(set(df['Distrito'].unique())-set(df_medias['Distrito'].unique()))
print( "Relação de Distritos sem valores de Latitude e Longitude:\n\n", places)

In [ ]:
# Com a biblioteca Nominatim, encontra-se Latitude e Longitude e insere no df_medias:

places = list(set(df['Distrito'].unique())-set(df_medias['Distrito'].unique()))

geolocator = Nominatim(user_agent="geolocalização")
for place in places:
    location = geolocator.geocode( place+'- São Paulo - SP')
    df_medias = df_medias.append({'Distrito' : place , 'Latitude' : location.latitude,
                                  'Longitude' : location.longitude}, ignore_index=True)
    
df_medias.tail()

In [ ]:
# No dataframe df, substitui as latitudes e longitudes incorretas pelo nome do respectivo distrito
df_medias.rename(columns={'Latitude':'media_Latitude', 'Longitude': 'media_Longitude'}, inplace=True)
df = pd.merge(df, df_medias, on='Distrito', how='left')

df["Latitude"].fillna(df['media_Latitude'], inplace=True)
df["Longitude"].fillna(df['media_Longitude'], inplace=True)

df.drop(columns = ['media_Latitude', 'media_Longitude'], inplace = True)
print('Total registros de Latitude:', df.Latitude.count())
print('Total registros de Longitude:', df.Longitude.count())


In [ ]:
m = folium.Map(
    location=[-23.5, -46.6],
    tiles='Stamen Toner',
    zoom_start=8
)
mc = MarkerCluster()
    
for index, value in df.iterrows():
    mc.add_child(folium.Marker([value['Latitude'], value['Longitude']], 
              popup=str(value['Distrito']),
              tooltip=value['Distrito'],
              icon=folium.Icon(icon='book'))).add_to(m)  
m

In [ ]:
coordenadas=[]
for lat,lng in zip(df.Latitude.values,df.Longitude.values):
    coordenadas.append([lat,lng])
m = folium.Map(location=[-23.55,-46.63], zoom_start=10)
m.add_child(plugins.HeatMap(coordenadas))        
m

In [ ]:
lat_zero = len(df[['Distrito','Latitude', 'Longitude']].loc[df['Latitude']==0])
long_zero = len(df[['Distrito','Latitude', 'Longitude']].loc[df['Longitude']==0])

print('Total de registros de Latitude iguais a zero:  ',lat_zero)
print('Total de registros de Longitude iguais a zero: ',long_zero)

In [ ]:
sns.set(font_scale = 1.5)
ax = sns.pairplot(df, y_vars = 'Latitude', x_vars = ['Longitude'],
                  height = 5, kind = 'reg')

ax.fig.suptitle('Dispersão entre as Variáveis', fontsize=2, y=0.5)

In [ ]:
# Exclusão de dados duplicados

df = df.drop_duplicates()
df.reset_index(drop=True, inplace=True)
display(df.shape)


In [ ]:
df_orig = df.copy()
df_orig.to_csv("dataset_consolidado.csv")

In [ ]:
#df = df_orig.copy()

In [ ]:
df.info()

## Variáveis numéricas

In [ ]:
df.dropna(inplace=True)
df = df.drop_duplicates()

df.reset_index(drop = True,inplace=True)
display(df.round().describe())
#display(df.describe())
print('Total de valores NaN: ', df.isna().sum().sum())
print('Dimensão do Dataframe: ', df.shape)

In [ ]:
# Convertendo variáveis do tipo object para inteiro
## chama função para relacionar as colunas numéricas e categóricas
col = df.columns
col_object=[]
col_numeric=[]
for i in col:
    if df[i].dtypes in [np.object]:
        col_object.append(i)
    elif df[i].dtypes in [np.int64, np.float64]:
        col_numeric.append(i)

print('Variáveis tipo object: ', col_object)

for col in col_object:
    df[col] = pd.Categorical(df[col])
    df[col] = df[col].cat.codes

df.Distrito.describe()

# Tratamento variáveis Numéricas
# Correlação das variáveis - Baixa correlação e Multicolinearidade

In [ ]:
df.info()

# Baixa correlação

In [ ]:
#A variável Preco e a correlação com as demais variáveis

df.corr()['Preco'].sort_values().plot(kind = 'bar',yticks = [-.9,-.1,0,.1,.9], mark_right=False, figsize=(14,5))

In [ ]:
# Selecionando as variáveis com correlação absoluta maior que |0.1|

s_corr = df.corr()['Preco'].abs()

s_corr = s_corr.loc[s_corr>0.1]
s_corr.sort_values(ascending=False)
l_select_cols = list(s_corr.index)

print('Variáveis consideradas para verificar multicolinearidade:', l_select_cols)

In [ ]:
# Plotando o mapa de correlação entre as variáveis
sns.set(font_scale = 1)
plt.figure(figsize=(14,10))
plt.title("Matriz de Correlação", fontsize=20)
sns.heatmap(df[l_select_cols].corr(), cbar=True, square= True, fmt='.2f', annot=True, cmap='viridis')

In [ ]:
# Multicolinearidade - verificando quais as variáveis preditoras estão correlacionadas

df_corr = df[l_select_cols].corr()
df_corr = pd.DataFrame(df_corr)
df_corr.head()

In [ ]:
# Seleção das variáveis preditoras correlacionadas e criação do dataframe com a respectiva correlação com Preco
df_corr['indice'] = df_corr.index
df_corr = df_corr.melt(id_vars='indice')
df_corr = df_corr.sort_values(by = 'indice',
                                  ascending=False)[(abs(round(df_corr['value'],
                                                              2))>=.90)].loc[df_corr['indice']!= df_corr['variable']]

columns = list(df_corr['indice'].unique())
columns.append('Preco')
df_corr_preco = pd.DataFrame(df[columns].corr()['Preco'].abs()).reset_index().sort_values(by='Preco',ascending=False)
df_corr_preco

In [ ]:
# Dentre essas variáveis, serão excluídas aquelas de menor grau de correlação em relação a variável Preco.
# Com a função .merge() serão acrescentadas colunas referentes a variável e sua correlação com a preco
df_corr = pd.merge(df_corr, df_corr_preco, how = 'left', left_on='indice', right_on = 'index')
df_corr = pd.merge(df_corr, df_corr_preco, how = 'left', left_on='variable', right_on = 'index')
df_corr.head()

In [ ]:
# Agora são criadas colunas que irão selecionar quais as que tem maior correlação e as de menor correlação
df_corr["manter"] = np.where(df_corr['Preco_x']>df_corr['Preco_y'],df_corr['index_x'], df_corr['index_y'])
df_corr["excluir"] = np.where(df_corr['Preco_x']<df_corr['Preco_y'],df_corr['index_x'], df_corr['index_y'])
df_corr.head()

In [ ]:
# A seguir é aplicado método para listar quais as variáveis que devem ser mantidas e quais devem ser excluídas
# as variáveis a serem excluidas serão subtraídas da lista de variáveis que tinham correlação maior que 0.1

l_manter = list(set(df_corr["manter"].unique())-set(df_corr["excluir"].unique()))
l_excluir = list(set(df_corr["excluir"].unique())-set(l_manter))
l_select_cols = list(set(l_select_cols)-set(l_excluir))
print(l_manter)
print('Colunas mantidas:', l_select_cols,'\n\nColunas a serem excluídas por Multicolinearidade:',l_excluir)


In [ ]:
# Plotando o mapa de correlação entre as variáveis
l_select_cols = list(df[l_select_cols].corr()['Preco'].abs().sort_values(ascending=False).index)

sns.set(font_scale = 1.5)
plt.figure(figsize=(14,11))
plt.title("Matriz de Correlação", fontsize=20)
sns.heatmap(df[l_select_cols].corr(), cbar=True, square= True, fmt='.2f', annot=True,
            annot_kws={'size':15}, cmap='viridis')

In [ ]:
# Selecionando variáveis da lista l_select_cols:

df=df[l_select_cols]

In [ ]:
l_select_cols

In [ ]:
#df=df_orig[l_select_cols].copy()

# Análise variável alvo: Preço

In [ ]:
sns.set(font_scale = 2)
plt.figure(figsize=(12,4))
print(sns.boxplot(data=df, x='Preco'))

sns.set(font_scale = 2)
plt.figure(figsize=(12,4))
sns.displot(data=df, x='Preco', kde=True,height=8)

df['Preco'].describe().round(2).astype(str)

In [ ]:
# Criando novo dataframe com base logatmica para variáveis com distribuição assimétrica

df_ln = df.copy()
cols_ln = simetric(df_ln, l_select_cols)

In [ ]:
df_ln.info()

In [ ]:
sns.set(font_scale = 2)
plt.figure(figsize=(12,4))
print(sns.boxplot(data=df_ln, x='Preco'))

sns.set(font_scale = 2)
plt.figure(figsize=(12,4))
sns.displot(data=df_ln, x='Preco', kde=True,height=8)

df_ln['Preco'].describe().round(2).astype(str)

In [ ]:
# Plotando o mapa de correlação entre as variáveis
df_ln=df_ln[['Preco', 'Area', 'Garagem', 'Condo', 'Banheiros',
        'Mais de 20_SM_%','Quartos', 'Longitude','População (2010)', 'Piscina','Densidade Demográfica (Hab/km²)']]

sns.set(font_scale = 1.5)
plt.figure(figsize=(15,12))
plt.title("Matriz de Correlação", fontsize=20)
sns.heatmap(df_ln.corr(), cbar=True, square= True, fmt='.2f', annot=True, annot_kws={'size':15}, cmap='viridis')

# Tratamento de Outliers

In [ ]:
# A variável Area é a que mais influencia no Preco.
df_out = df.copy()
df_ln_out = df_ln.copy()

df['Preco_Area'] = round(df['Preco'] / df['Area'],2)
df_ln['Preco_Area'] = round(df_ln['Preco'] / df_ln['Area'],2)
 
sns.displot(data=df, x='Preco_Area', kde=True,height=4),
sns.displot(data=df_ln, x='Preco_Area', kde=True,height=4)

In [ ]:
plt.figure(figsize=(12,3))
print(sns.boxplot(data=df, x='Preco_Area'))

In [ ]:
plt.figure(figsize=(12,3))
print(sns.boxplot(data=df_ln, x='Preco_Area'))

In [ ]:
# Aplicação da função outlier_iqr() 
cols = ['Preco_Area']
low_out,upp_out = outlier_iqr(df, cols)

cols = ['Preco_Area']
low_out_ln,upp_out_ln = outlier_iqr(df_ln, cols)

In [ ]:
# Cálculo do Intervalo Interquatílico (IQR) na variável Preco_Area nos datasets df e df_ln:

df = df[df['Preco_Area'] > low_out]
df = df[df['Preco_Area'] < upp_out]

df_ln = df_ln[df_ln['Preco_Area'] > low_out_ln]
df_ln = df_ln[df_ln['Preco_Area'] < upp_out_ln]



df.drop(columns = ['Preco_Area'], inplace = True)
df_ln.drop(columns = ['Preco_Area'], inplace = True)

print('Shape do df:', df.shape)
print('Shape do df_ln:', df_ln.shape)

In [ ]:
# Aplicando RobustScaler em df e df_out
cols = list(df.columns)
df_r = RobustScaler().fit_transform(df)
df_r = pd.DataFrame(df_r, columns=cols)
df= df_r.copy()

cols = list(df_out.columns)
df_r = RobustScaler().fit_transform(df_out)
df_r = pd.DataFrame(df_r, columns=cols)
df_out = df_r.copy()

   
df_out.head()

In [ ]:
# Excluindo dados duplicados

df_ = df.drop_duplicates()
df_ln = df_ln.drop_duplicates()
df_out = df_out.drop_duplicates()
df_ln_out = df_ln_out.drop_duplicates()

#Verificando se há dados nulos
print('Totais de dados nulos\ndf: %s \ndf_ln: %s \ndf_out: %s  \ndf_ln_out: %s'
      '\n\nDimensão dos DataFrames\ndf: %s \ndf_ln: %s \ndf_out: %s \ndl_ln_out:%s'
      %(df_.isna().sum().sum(), df_ln.isna().sum().sum(), df_out.isna().sum().sum(), df_ln_out.isna().sum().sum(),
        df.shape,df_ln.shape, df_out.shape, df_ln_out.shape))

In [ ]:
df_.info()

In [ ]:
df_.to_csv('dataset_processado_SP.csv')
df_ln.to_csv('dataset_processado_SP_ln.csv')
df_out.to_csv('dataset_processado_SP_out.csv')
df_ln_out.to_csv('dataset_processado_SP_ln_out.csv')

# CRIAÇÃO E AVALIAÇÃO DE MODELOS DE MACHINE LEARNING

In [ ]:
#df_ = pd.read_csv('dataset_processado_SP.csv', index_col=0)
#df_ln = pd.read_csv('dataset_processado_SP_ln.csv', index_col=0)
#df_out = pd.read_csv('dataset_processado_SP_out.csv', index_col=0)
#df_ln_out = pd.read_csv('dataset_processado_SP_ln_out.csv', index_col=0)

In [ ]:
def Reg_Linear(dataset,dataframe,modelos, scores_map, kf, rs):
    x = dataframe.drop(columns='Preco')
    y = dataframe['Preco']   
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2,random_state=rs)
    parameters = {'n_jobs':[-1,1,2,3,5,10,25], "fit_intercept": [True, False],'normalize':[False, True]}
    # define modelo/ estimador
    model = LinearRegression()    
    # definindo Rand search
    rand_cv= RandomizedSearchCV(model, param_distributions=parameters, scoring='neg_mean_squared_error',
                                cv=kf, random_state=rs)    
    #fit no Rand search
    rand_cv.fit(train_x,train_y)   
    scores = cross_val_score(model, x, y, cv=kf)   
    d_scores={}
    d_scores['Regressão_Linear'] = scores
    scores_map[dataset].update(d_scores)
   
    # melhor estimador
    print("Melhor classificação :", rand_cv.best_estimator_)    
    # melhor modelo
    modelo = rand_cv.best_estimator_
    modelo.fit(train_x,train_y)

    y_pred = modelo.predict(train_x)
    # Calcula a métrica Root Mean Squared Error
    rmse_train=sqrt(mean_squared_error(train_y,y_pred))  
    #Calcula a métrica Mean Absolute Error
    mae_train = mean_absolute_error(train_y,y_pred)
    score_train = modelo.score(train_x, train_y)*100    
    
    modelo.fit(train_x,train_y)
    y_pred = modelo.predict(test_x)   
    rmse_test=sqrt(mean_squared_error(test_y,y_pred))
    mae_test = mean_absolute_error(test_y,y_pred)    
    score_test = modelo.score(test_x, test_y)*100   
    modelos[dataset].append(modelo)
    resultados = {'Dataframe': [dataset], 'Modelo': [modelo], 'MAE - Treina':[mae_train], 'MAE - Teste':[mae_test],
                  'RMSE - Treina': [rmse_train],'RMSE - Teste': [rmse_test],
                  'R2 (Acurácia) - Treina':[score_train],'R2 (Acurácia) - Teste': [score_test]} 
    
    return modelos, scores_map,resultados

In [ ]:
# Função Ridge
def Ridge_(dataset,dataframe,modelos, scores_map, kf, rs):
    x = dataframe.drop(columns='Preco')
    y = dataframe['Preco']   
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2,random_state=rs)
    parameters = {'alpha':[0.0001,0.001,0.01,0.05,0.1,1, 10,100],'max_iter':[10,20,50,100,200,300,500,700,1000,2000,5000,10000],
                  'normalize':[False, True], 'fit_intercept':[False, True]}
    # define modelo/ estimador
    model = Ridge()
    
    # definindo Rand search
    rand_cv= RandomizedSearchCV(model, param_distributions=parameters,scoring='neg_mean_squared_error',
                                cv=kf, random_state=rs)
    
    #fit no Rand search
    rand_cv.fit(train_x,train_y)    
    scores = cross_val_score(model, x, y, cv=kf)
    d_scores={}
    d_scores['Ridge'] = scores
    scores_map[dataset].update(d_scores)
    # melhor estimador
    print("Melhor classificação :", rand_cv.best_estimator_)   
    # melhor modelo
    modelo = rand_cv.best_estimator_
    modelo.fit(train_x,train_y)

    y_pred = modelo.predict(train_x)
    # Calcula a métrica Root Mean Squared Error
    rmse_train=sqrt(mean_squared_error(train_y,y_pred))  
    #Calcula a métrica Mean Absolute Error
    mae_train = mean_absolute_error(train_y,y_pred)
    score_train = modelo.score(train_x, train_y)*100    
    
    modelo.fit(train_x,train_y)
    y_pred = modelo.predict(test_x)   
    rmse_test=sqrt(mean_squared_error(test_y,y_pred))
    mae_test = mean_absolute_error(test_y,y_pred)    
    score_test = modelo.score(test_x, test_y)*100   
    modelos[dataset].append(modelo)
    resultados = {'Dataframe': [dataset], 'Modelo': [modelo], 'MAE - Treina':[mae_train], 'MAE - Teste':[mae_test],
                  'RMSE - Treina': [rmse_train],'RMSE - Teste': [rmse_test],
                  'R2 (Acurácia) - Treina':[score_train],'R2 (Acurácia) - Teste': [score_test]} 
    
    return modelos, scores_map,resultados

In [ ]:
def Lasso_(dataset,dataframe,modelos, scores_map, kf, rs):
    x = dataframe.drop(columns='Preco')
    y = dataframe['Preco']       
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2,random_state=rs)
    parameters = {'alpha':[0.0001,0.001,0.01,0.05,0.1,1, 10,100],'max_iter':[10,20,50,100,200,300,500,700,1000,2000,5000,10000],
                  'normalize':[False, True], 'fit_intercept':[False, True]}
    
    # define modelo/ estimador
    model = Lasso()    
    # definindo Rand search
    rand_cv= RandomizedSearchCV(model, param_distributions=parameters,scoring='neg_mean_squared_error',
                                cv=kf, random_state=rs)  
    #fit no Rand search
    rand_cv.fit(train_x,train_y) 
    scores = cross_val_score(model, x, y, cv=kf)
    d_scores = {}
    scores_map['Lasso'] = scores
    scores_map[dataset].update(d_scores)
    print(scores)
    # melhor estimador
    print("Melhor classificação :", rand_cv.best_estimator_)
    
    # melhor modelo
    modelo = rand_cv.best_estimator_
    modelo.fit(train_x,train_y)

    y_pred = modelo.predict(train_x)
    # Calcula a métrica Root Mean Squared Error
    rmse_train=sqrt(mean_squared_error(train_y,y_pred))  
    #Calcula a métrica Mean Absolute Error
    mae_train = mean_absolute_error(train_y,y_pred)
    score_train = modelo.score(train_x, train_y)*100    
    
    modelo.fit(train_x,train_y)
    y_pred = modelo.predict(test_x)   
    rmse_test=sqrt(mean_squared_error(test_y,y_pred))
    mae_test = mean_absolute_error(test_y,y_pred)    
    score_test = modelo.score(test_x, test_y)*100    
    modelos[dataset].append(modelo)
    resultados = {'Dataframe': [dataset], 'Modelo': [modelo], 'MAE - Treina':[mae_train], 'MAE - Teste':[mae_test],
                  'RMSE - Treina': [rmse_train],'RMSE - Teste': [rmse_test],
                  'R2 (Acurácia) - Treina':[score_train],'R2 (Acurácia) - Teste': [score_test]} 
    
    return modelos, scores_map,resultados

In [ ]:
def Elastic_Net(dataset,dataframe,modelos, scores_map, kf, rs):
    x = dataframe.drop(columns='Preco')
    y = dataframe['Preco']
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2,random_state=rs)
    parameters = {'alpha':[0.0001,0.001,0.01,0.05,0.1,1, 10,100],'max_iter':[10,20,50,100,200,300,500,700,1000,2000,5000,10000],
                  'l1_ratio': np.arange(0.0, 1.0, 0.1),'fit_intercept':[False, True],'normalize':[False, True]}
    # define modelo/ estimador
    model = ElasticNet()
    
    # definindo Rand search
    rand_cv= RandomizedSearchCV(model, param_distributions= parameters,cv=kf, random_state=rs)   
    #fit no Rand search
    rand_cv.fit(train_x,train_y)
    
    scores = cross_val_score(model, x, y, cv=kf)
    d_scores={}
    d_scores['ElasticNet'] = scores
    scores_map[dataset].update(d_scores)
    
    # melhor estimador
    print("Melhor classificação :", rand_cv.best_estimator_)    
    # melhor modelo
    modelo = rand_cv.best_estimator_
    modelo.fit(train_x,train_y)

    y_pred = modelo.predict(train_x)
    # Calcula a métrica Root Mean Squared Error
    rmse_train=sqrt(mean_squared_error(train_y,y_pred))  
    #Calcula a métrica Mean Absolute Error
    mae_train = mean_absolute_error(train_y,y_pred)
    score_train = modelo.score(train_x, train_y)*100    
    
    modelo.fit(train_x,train_y)
    y_pred = modelo.predict(test_x)   
    rmse_test=sqrt(mean_squared_error(test_y,y_pred))
    mae_test = mean_absolute_error(test_y,y_pred)    
    score_test = modelo.score(test_x, test_y)*100
    modelos[dataset].append(modelo)
    resultados = {'Dataframe': [dataset], 'Modelo': [modelo], 'MAE - Treina':[mae_train], 'MAE - Teste':[mae_test],
                  'RMSE - Treina': [rmse_train],'RMSE - Teste': [rmse_test],
                  'R2 (Acurácia) - Treina':[score_train],'R2 (Acurácia) - Teste': [score_test]} 
    
    return modelos, scores_map,resultados

In [ ]:
def Random_Forest_Regressor(dataset,dataframe,modelos, scores_map, kf, rs):
    x = dataframe.drop(columns='Preco')
    y = dataframe['Preco']
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2,random_state=rs)
    
    parameters={'n_estimators':[50,100,150,200], 'max_depth':[1,2,3,5,6,7,8,9,10,15,20,None],
                'min_samples_leaf':randint(32,128), "min_samples_split":randint(32,128)}  
    # define modelo/ estimador
    model = RandomForestRegressor()   
    # definindo Rand search
    rand_cv= RandomizedSearchCV(model, param_distributions=parameters,
                                scoring='neg_mean_squared_error',cv=kf, random_state=rs)
    #fit no Rand search
    rand_cv.fit(train_x,train_y)
    
    scores = cross_val_score(model, x, y, cv=kf)    
    d_scores={}
    d_scores['RandomForestReg'] = scores
    scores_map[dataset].update(d_scores)
    
    # melhor estimador
    print("Melhor classificação :", rand_cv.best_estimator_)    
    # melhor modelo
    modelo = rand_cv.best_estimator_
    modelo.fit(train_x,train_y)

    y_pred = modelo.predict(train_x)
    # Calcula a métrica Root Mean Squared Error
    rmse_train=sqrt(mean_squared_error(train_y,y_pred))  
    #Calcula a métrica Mean Absolute Error
    mae_train = mean_absolute_error(train_y,y_pred)
    score_train = modelo.score(train_x, train_y)*100    
    
    modelo.fit(train_x,train_y)
    y_pred = modelo.predict(test_x)   
    rmse_test=sqrt(mean_squared_error(test_y,y_pred))
    mae_test = mean_absolute_error(test_y,y_pred)    
    score_test = modelo.score(test_x, test_y)*100     
    modelos[dataset].append(modelo)
    resultados = {'Dataframe': [dataset], 'Modelo': [modelo], 'MAE - Treina':[mae_train], 'MAE - Teste':[mae_test],
                  'RMSE - Treina': [rmse_train],'RMSE - Teste': [rmse_test],
                  'R2 (Acurácia) - Treina':[score_train],'R2 (Acurácia) - Teste': [score_test]}  
    
    return modelos, scores_map,resultados

In [ ]:
def Decision_Tree_Regressor(dataset,dataframe,modelos, scores_map, kf, rs):
    x = dataframe.drop(columns='Preco')
    y = dataframe['Preco']
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2,random_state=rs)   
    parameters = {'max_depth':[3,5,10,15,20,30,None],'min_samples_leaf':randint(32, 128),
                  "min_samples_split":randint(32,128)}
    
    model = DecisionTreeRegressor()
    rand_cv = RandomizedSearchCV(model, cv=kf, param_distributions=parameters,
                                 scoring='neg_mean_squared_error', random_state=rs)
    
    #fit no Rand search
    rand_cv.fit(train_x, train_y)
    print("Melhor classificação :", rand_cv.best_estimator_)
    scores = cross_val_score(model,x, y, cv=kf)
    d_scores={}
    d_scores['DecisionTreeReg'] = scores
    scores_map[dataset].update(d_scores)
    # melhor modelo
    
    modelo = rand_cv.best_estimator_
    modelo.fit(train_x,train_y)

    y_pred = modelo.predict(train_x)
    # Calcula a métrica Root Mean Squared Error
    rmse_train=sqrt(mean_squared_error(train_y,y_pred))  
    #Calcula a métrica Mean Absolute Error
    mae_train = mean_absolute_error(train_y,y_pred)
    score_train = modelo.score(train_x, train_y)*100    
    
    modelo.fit(train_x,train_y)
    y_pred = modelo.predict(test_x)   
    rmse_test=sqrt(mean_squared_error(test_y,y_pred))
    mae_test = mean_absolute_error(test_y,y_pred)    
    score_test = modelo.score(test_x, test_y)*100
   
    modelos[dataset].append(modelo)
    resultados = {'Dataframe': [dataset], 'Modelo': [modelo], 'MAE - Treina':[mae_train], 'MAE - Teste':[mae_test],
                  'RMSE - Treina': [rmse_train],'RMSE - Teste': [rmse_test],
                  'R2 (Acurácia) - Treina':[score_train],'R2 (Acurácia) - Teste': [score_test]}
    
    return modelos, scores_map,resultados


In [ ]:
def K_Neighbors_Regressor(dataset,dataframe,modelos, scores_map, kf, rs):
    x = dataframe.drop(columns='Preco')
    y = dataframe['Preco']
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2,random_state=rs)
    parameters = {"n_neighbors" : [2,3,4,5,6,7,8,9,10,12,15],'weights':['uniform', 'distance'],
                  'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'], 'p':[1,2]}
    
    model = KNeighborsRegressor()
    rand_cv = RandomizedSearchCV(model, cv=kf, param_distributions=parameters,
                                 scoring='neg_mean_squared_error', random_state=rs)   
    #fit no Rand search
    rand_cv.fit(train_x, train_y)
    
    scores = cross_val_score(model, x, y, cv=kf)
    d_scores={}
    d_scores['KNeighborsReg'] = scores 
    scores_map[dataset].update(d_scores)
    
    print("Melhor classificação :", rand_cv.best_estimator_)
    # melhor modelo
    modelo = rand_cv.best_estimator_
    modelo.fit(train_x,train_y)

    y_pred = modelo.predict(train_x)
    # Calcula a métrica Root Mean Squared Error
    rmse_train=sqrt(mean_squared_error(train_y,y_pred))  
    #Calcula a métrica Mean Absolute Error
    mae_train = mean_absolute_error(train_y,y_pred)
    score_train = modelo.score(train_x, train_y)*100    
    
    modelo.fit(train_x,train_y)
    y_pred = modelo.predict(test_x)   
    rmse_test=sqrt(mean_squared_error(test_y,y_pred))
    mae_test = mean_absolute_error(test_y,y_pred)    
    score_test = modelo.score(test_x, test_y)*100    
    modelos[dataset].append(modelo)
    resultados = {'Dataframe': [dataset], 'Modelo': [modelo], 'MAE - Treina':[mae_train], 'MAE - Teste':[mae_test],
                  'RMSE - Treina': [rmse_train],'RMSE - Teste': [rmse_test],
                  'R2 (Acurácia) - Treina':[score_train],'R2 (Acurácia) - Teste': [score_test]} 
    
    return modelos, scores_map,resultados

In [ ]:
def Gradient_Boosting_Regressor(dataset,dataframe,modelos, scores_map, kf, rs):
    dataframe.dropna(inplace=True)
    x = dataframe.drop(columns='Preco')
    y = dataframe['Preco']
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2,random_state=rs)
    
    model = GradientBoostingRegressor()
    parameters={'n_estimators':[50,100, 150,200,250],
                'learning_rate':[0.001,0.01,0.05,0.1,1,5,10],
                "min_samples_split" : randint(32, 128),  "max_depth" : [3, 5, 10, 15, 20, 30, None],
                "min_samples_leaf" : randint(32, 128)}
    
    rand_cv = RandomizedSearchCV(model, cv=kf, param_distributions=parameters,
                                 scoring='neg_mean_squared_error', random_state=rs)   
    #fit no Rand search
    rand_cv.fit(train_x, train_y)
    print("Melhor classificação:", rand_cv.best_estimator_)
    scores = cross_val_score(model, x, y, cv=kf)
    d_scores={}
    d_scores['GradientBoostingReg'] = scores
    scores_map[dataset].update(d_scores)
    
    # melhor modelo
    modelo = rand_cv.best_estimator_
    modelo.fit(train_x,train_y)

    y_pred = modelo.predict(train_x)
    # Calcula a métrica Root Mean Squared Error
    rmse_train=sqrt(mean_squared_error(train_y,y_pred))  
    #Calcula a métrica Mean Absolute Error
    mae_train = mean_absolute_error(train_y,y_pred)
    score_train = modelo.score(train_x, train_y)*100    
    
    modelo.fit(train_x,train_y)
    y_pred = modelo.predict(test_x)   
    rmse_test=sqrt(mean_squared_error(test_y,y_pred))
    mae_test = mean_absolute_error(test_y,y_pred)    
    score_test = modelo.score(test_x, test_y)*100 
    
    modelos[dataset].append(modelo)
    resultados = {'Dataframe': [dataset], 'Modelo': [modelo], 'MAE - Treina':[mae_train], 'MAE - Teste':[mae_test],
                  'RMSE - Treina': [rmse_train],'RMSE - Teste': [rmse_test],
                  'R2 (Acurácia) - Treina':[score_train],'R2 (Acurácia) - Teste': [score_test]}  
    
    return modelos, scores_map,resultados

In [ ]:
def AdaBoost_Regressor(dataset,dataframe,modelos, scores_map, kf, rs):
    dataframe.dropna(inplace=True)
    x = dataframe.drop(columns='Preco')
    y = dataframe['Preco']
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2,random_state=rs)
    
    model = AdaBoostRegressor()
    parameters={'n_estimators':[50,100, 150,200,250],
                'learning_rate':[0.001,0.01,0.05,0.1,1,5,10,100],
                'loss':['linear', 'square', 'exponential']}
    rand_cv = RandomizedSearchCV(model, cv=kf,param_distributions= parameters,
                                 scoring='neg_mean_squared_error', random_state=rs)   
    #fit no Rand search
    rand_cv.fit(train_x, train_y)
    print("Melhor classificação:", rand_cv.best_estimator_)
    scores = cross_val_score(model, x, y, cv=kf)
    d_scores={}    
    d_scores['AdaBoostReg'] = scores
    scores_map[dataset].update(d_scores)
    
    # melhor modelo
    modelo = rand_cv.best_estimator_
    modelo.fit(train_x,train_y)

    y_pred = modelo.predict(train_x)
    # Calcula a métrica Root Mean Squared Error
    rmse_train=sqrt(mean_squared_error(train_y,y_pred))  
    #Calcula a métrica Mean Absolute Error
    mae_train = mean_absolute_error(train_y,y_pred)
    score_train = modelo.score(train_x, train_y)*100    
   
    modelo.fit(train_x,train_y)
    y_pred = modelo.predict(test_x)   
    rmse_test=sqrt(mean_squared_error(test_y,y_pred))
    mae_test = mean_absolute_error(test_y,y_pred)    
    score_test = modelo.score(test_x, test_y)*100
    
    modelos[dataset].append(modelo)
    
    resultados = {'Dataframe': [dataset], 'Modelo': [modelo], 'MAE - Treina':[mae_train], 'MAE - Teste':[mae_test],
                  'RMSE - Treina': [rmse_train],'RMSE - Teste': [rmse_test], 'R2 (Acurácia) - Treina':[score_train],
                  'R2 (Acurácia) - Teste': [score_test]}  
    
    return modelos, scores_map,resultados

# Aplicando as funções de ML

In [ ]:
modelos={'df':[],'df_ln':[],'df_out':[],'df_ln_out':[]}
scores_map = {'df':{},'df_ln':{},'df_out':{},'df_ln_out':{}}
kf = KFold(n_splits=5, shuffle=True)
rs = 2000

ml_functions = ['K_Neighbors_Regressor', 'Random_Forest_Regressor','Decision_Tree_Regressor', 'Reg_Linear',
                'Ridge_', 'Lasso_','Elastic_Net', 'AdaBoost_Regressor', 'Gradient_Boosting_Regressor']

datasets = {'df':df, 'df_ln':df_ln, 'df_out':df_out, 'df_ln_out':df_ln_out}


df_compara = pd.DataFrame(columns = ['Dataframe', 'Modelo', 'MAE - Treina', 'MAE - Teste', 'RMSE - Treina',
                                    'RMSE - Teste', 'R2 (Acurácia) - Treina', 'R2 (Acurácia) - Teste'])

for function in ml_functions:    
    for dataset,dataframe in datasets.items():
            modelos, scores_map,resultados = globals()[function](dataset,dataframe,modelos, scores_map, kf,rs)
            df_mod=pd.DataFrame(data = resultados)
            df_compara=pd.concat([df_compara,df_mod])

In [ ]:
df_compara.sort_values(by=['R2 (Acurácia) - Teste'], ascending=False, inplace=True)
d_modelo = {df_compara.iloc[0, 0]: df_compara.iloc[0, 5]}
df_compara['Modelos'] = df_compara['Modelo'].astype(str)
df_compara[['Dataframe', 'MAE - Treina', 'MAE - Teste', 'RMSE - Treina',
            'RMSE - Teste', 'R2 (Acurácia) - Treina', 'R2 (Acurácia) - Teste','Modelos']].head(30)

In [ ]:
df_compara.to_csv('resultados02-4.csv')

# Selecionando os modelos

In [ ]:
d_dfs_scores={}
d_dfs_scores['Grad_Boost_df_ln'] = scores_map['df_ln']['GradientBoostingReg']
d_dfs_scores['GradBoost_dfln_out'] = scores_map['df_ln_out']['GradientBoostingReg']
d_dfs_scores['DecTree_df_ln'] = scores_map['df_ln']['DecisionTreeReg']
d_dfs_scores['Grad_Boost_df'] = scores_map['df']['GradientBoostingReg']

d_dfs_scores['RandomForest_df_ln'] = scores_map['df_ln']['RandomForestReg']
d_dfs_scores['KNN_Regr_df_ln'] = scores_map['df_ln']['KNeighborsReg']


plt.figure(figsize=(20, 12))
df_scores_map = pd.DataFrame(d_dfs_scores)
sns.boxplot(data=df_scores_map)

# Análise de resíduos - Melhor modelo

In [ ]:
#Relação de modelos
modelo_sel = [df_compara.iloc[0, 1], df_compara.iloc[5, 1]]

df_ln_pred = df_ln.copy()

modelo=modelo_sel[0]

x = df_ln.drop(columns='Preco')
y = df_ln['Preco']
# Treinando o modelo
modelo.fit(x,y)
y_pred = modelo.predict(x)

df_ln_pred['preco_predito'] = y_pred

# Calcula a métrica Root Mean Squared Error
rmse=round(sqrt(mean_squared_error(y,y_pred)),4)
#Calcula a métrica Mean Absolute Error
mae = round(mean_absolute_error(y,y_pred),4)
score = round(modelo.score(x, y),4)

print("Dataframe: df_ln\nModelo: GradientBoostingRegressor\nMAE:%s \nRMSE: %s\nR²:%s" %(mae,rmse, score))
plotar_grafico(y,y_pred)



# Comparação resultados

In [ ]:
modelo=modelo_sel[1]

x = df_ln.drop(columns='Preco')
y = df_ln['Preco']

# Treinando o modelo
modelo.fit(x,y)
y_pred = modelo.predict(x)

# Calcula a métrica Root Mean Squared Error
rmse=round(sqrt(mean_squared_error(y,y_pred)),4)
#Calcula a métrica Mean Absolute Error
mae = round(mean_absolute_error(y,y_pred),4)
score = round(modelo.score(x, y),4)

print("Dataframe: df_ln\nModelo: RandomForestRegressor\nMAE:%s \nRMSE: %s\nR²:%s" %(mae,rmse, score))

plotar_grafico(y,y_pred)


# Apresentação de Resultado
## Precisão por distritos e preços médios

In [ ]:
df_ln_pred.rename(columns={'Preco':'Preco_ln'}, inplace=True)

df_results = df_orig[df_orig.index.isin(df_ln_pred.index)]

df_results = pd.concat([df_results,df_ln_pred[['Preco_ln','preco_predito']]], axis=1)

df_ln_pred.rename(columns={'Preco_ln':'Preco'}, inplace=True)
df_ln_pred.drop(columns=['preco_predito'], inplace=True)

df_results['preco_predito_x'] = np.exp(df_results['preco_predito'])
df_results['Precisao'] = np.where(df_results['preco_predito_x']/df_results['Preco']>1,
                                 df_results['Preco']/df_results['preco_predito_x']*100,
                                  df_results['preco_predito_x']/df_results['Preco']*100)

df_results.shape


In [ ]:
# Distribuição dos imóveis na cidade de São Paulo

grouped = df_results.groupby('Distrito')['Precisao'].mean().reset_index()

df_group = grouped.copy()

grouped = grouped.sort_values(by = 'Precisao', ascending = True).head()

plt.figure(figsize = (5,4))
sns.set(font_scale = 1.4)
ax = sns.histplot(x='Precisao', y='Distrito', data = grouped,bins=100)
print("Precisão: Distritos com as piores previsões")

In [ ]:
# Distribuição dos imóveis na cidade de São Paulo

grouped = df_results.groupby('Distrito')['Precisao'].mean().reset_index()

#df_group = grouped.copy()

grouped = grouped.sort_values(by = 'Precisao', ascending = False).head(5)

plt.figure(figsize = (5,4))
sns.set(font_scale = 1.40)
ax = sns.histplot(x='Precisao', y='Distrito', data = grouped, bins=100)
print("Precisão: Distritos com as melhores previsões")

In [ ]:
# Distribuição dos imóveis na cidade de São Paulo

grouped = df_results.groupby('Distrito')['Preco'].mean().reset_index()

df_group = pd.merge(df_group, grouped, how='inner', on='Distrito')

df_group.rename(columns={'Preco':'Preco_Medio'}, inplace=True)
df_group = pd.merge(df_group, df_medias, how='inner', on='Distrito')

df_group.to_csv('ap_resultado.csv')

In [ ]:
# Criando Regras de zona

df_group['Zona']= np.where(df_group.media_Longitude>df_group.media_Longitude.quantile(0.70),'Centro/Leste',
                                    np.where(df_group.media_Latitude<df_group.media_Latitude.median(),'Sul',
                                             'Centro/Norte'))
df_group['Zona']= np.where(df_group.media_Longitude>df_group.media_Longitude.quantile(0.80),'Leste',df_group['Zona'])

In [ ]:
df_group.describe()

# Mapa de resultados

In [ ]:
#Preparando Mapa de resultados

sul_group = folium.FeatureGroup(name='Sul')
leste_group = folium.FeatureGroup(name='Leste')
norte_group = folium.FeatureGroup(name='Centro/Norte')
centro_leste_group = folium.FeatureGroup(name='Centro/Leste')
m = folium.Map(location=[-23.55,-46.63], zoom_start=10)
ms = MarkerCluster()
ml = MarkerCluster()
mn = MarkerCluster()
mcl = MarkerCluster()
for dis,acur,preco,lat,long, zone in df_group.values:
    # coordinates to locate your marker
    COORDINATE = [lat,long]
    if zone == "Sul":
        ms.add_child(folium.Marker(location=COORDINATE,
                      popup=str(dis)+', Precisão '+str(round(acur,2))+ '%' +'. Preço Médio aptos R$ '+str(round(preco,2)),
                      tooltip=str(dis)+', Precisão '+str(round(acur,2))+ '%' +'. Preço Médio aptos: R$ '+str(round(preco,2)),
                      icon=folium.Icon(icon='book'))).add_to(sul_group)
    elif zone == "Leste":
        ml.add_child(folium.Marker(location=COORDINATE,
                      popup=str(dis)+', Precisão '+str(round(acur,2))+ '%' +'. Preço Médio aptos R$ '+str(round(preco,2)),
                      tooltip=str(dis)+', Precisão '+str(round(acur,2))+ '%' +'. Preço Médio aptos: R$ '+str(round(preco,2)),
                      icon=folium.Icon(icon='book'))).add_to(leste_group)
    elif zone == "Centro/Norte":
        mn.add_child(folium.Marker(location=COORDINATE,
                      popup=str(dis)+', Precisão '+str(round(acur,2))+ '%' +'. Preco Médio aptos R$ '+str(round(preco,2)),
                      tooltip=str(dis)+', Precisão '+str(round(acur,2))+ '%' +'. Preco Médio aptos: R$ '+str(round(preco,2)),
                      icon=folium.Icon(icon='book'))).add_to(norte_group)
    else:
        mcl.add_child(folium.Marker(location=COORDINATE,
                      popup=str(dis)+', Precisão '+str(round(acur,2))+ '%' +'. Preço Médio aptos R$ '+str(round(preco,2)),
                      tooltip=str(dis)+', Precisão '+str(round(acur,2))+ '%' +'. Preço Médio aptos: R$ '+str(round(preco,2)),
                      icon=folium.Icon(icon='book'))).add_to(centro_leste_group)

m.add_child(sul_group)
m.add_child(leste_group)
m.add_child(norte_group)
m.add_child(centro_leste_group)
# turn on layer control
m.add_child(folium.map.LayerControl(collapsed=False))
m.save("mapa_resultados.html")
m

In [ ]:
# Dicionário modelos. Chaves (nomes dos dataframes) e valor (modelo de ML com os respectivos melhores parâmetros calculados)
modelos

# FIM